In [1]:
# imports
import matplotlib.pyplot as plt
from os import path
import numpy as np
import scipy
import scipy.signal
from qutip import *
from helper_function_new import *
#from helper_function_L import *



In [2]:
##########################################################################
# in previous simulations, we ommited the lengths between the mirror and the qubits,
# and its affect on the system
# we want to see the effect of L (length of waveguide) on the simulation
#question:
# if we excite the qubits from the waveguide - is the phase created between the mirror and
# the qubit relevant? if not, do we ignore d_r in zm?
##########################################################################

# parameters
N = 6 # number of transmons in the model
c_light = 3E-1 #  0.3 m/ns
d = 3E-4 # 0.3 mm = 0.0003 m
eps = 6.5 # for silicon
L = 0.015 #15 mm

omega0 = 6 * (2*np.pi) # omega0 ~ 6 GHz
gamma0 = 0.03 * (2*np.pi) # gamma_1D ~ 30 MHz

omega,gamma = err_val_omega(0, omega0, gamma0, N, d,)
phi0 = np.round(omega0*d*np.sqrt(eps)/c_light,5) # phi ~ 0.1
k = np.round(omega0*np.sqrt(eps)/c_light,5) # phi = k*d


alpha = -0.25 * (2*np.pi) # 200 MHz
q_dim = 2 # Use only first two transmon levels
r_cavity = -0.99 # cavity/mirror reflection coefficient


In [ ]:
#########################################################################################################################################################
### Add driving: excitation of the desired two-excitation eigenstate.
### Plot probability of exciting desired two-excitation eigenstate as function of driving frequency for the case of driving via waveguide
#########################################################################################################################################################

w1_list = np.linspace(-1,1,81) # epsilon/gamma_{1D} in the range of [-1,1]
gamma0 = 0.005*omega0
Delta = 20*(gamma0/(2*np.pi)) # large detunning (Delta/2)/gamma = 10
E0 = 0.2*gamma0/(2*np.pi) # Pulse amplitude
q_dim = 2 # consider only two transmon levels
psi_init = tensor([basis(q_dim,0)]*N) # initial state of 6 transmons in ground state
tlist = np.linspace(0,15000,800)/(2*np.pi)
col_ops = [] # collapse operator represent here decay to enviorment, not to the waveguide
exp_ops = [] # don't include expectation value operators, calculate based on density matrix

P1state = [] # to hold probabilities of the state we are interested in
Ptots = [] # to hold total probability of two-excitation states


# N-dim sigma+ and sigma- operators for excitation through the waveguide - Em = e^(i*phi*m) + r*e^(i*phi*N)*e^(-i*k*zm)
Hwgexc_plus = construct_wg_driving_operator_L_plus(N, k, r_cavity, L, d, q_dim)
Hwgexc_minus = construct_wg_driving_operator_L_minus(N, k, r_cavity, L, d, q_dim)

H0 = construct_full_H(6,omega,d,gamma,-0.99)
evals_2exc,evecs_2exc = get_2excitations_states(N,H0,q_dim) # find two-excitation eigenstates

for wi in w1_list:

    args_exc = {'E':E0,'eps0':omega0/(2*np.pi)+wi*(gamma0/(2*np.pi)),'Delta0':Delta}# arguments for the exponent function describing qubit excitation

    # excitation through the waveguide
    Htot = [H0,[Hwgexc_plus,exc_f_plus],[Hwgexc_minus,exc_f_minus]]
    result = mesolve(Htot,psi_init,tlist,col_ops,exp_ops,args=args_exc)
    rho = result.states[-1]*result.states[-1].dag() # define density matrix rho
    P1state.append((rho*construct_2excitation_creation_operator(evecs_2exc[9],N,q_dim)*construct_2excitation_creation_operator(evecs_2exc[9],N,q_dim).dag()).tr())

    Ptot = 0
    for v in evecs_2exc:
        Ptot = Ptot + (rho*construct_2excitation_creation_operator(v,N,q_dim)*construct_2excitation_creation_operator(v,N,q_dim).dag()).tr()
    Ptots.append(Ptot)

plt.plot(w1_list, P1state,'b', label=r"$P_{state 10}$")
plt.plot(w1_list, Ptots,'r', label=r"$P_{tot}$")

plt.set_yscale('log')
plt.set_xlabel(r'$Re ~\left(\epsilon - \omega_{0} \right) / \gamma_{1D}$')
plt.set_ylabel('Excitation probability')
plt.set_title(r'N = 6, $\phi$ = 0.1, r = 0.99$e^{iN\phi}$ - excitation through waveguide')

plt.legend()
